주피터 노트북에 QIIME2 확장 설치

In [1]:
!jupyter serverextension enable --py qiime2 --sys-prefix

Enabling: qiime2.jupyter
- Writing config: /home/partrita/miniconda3/envs/qiime2-2019.7/etc/jupyter
    - Validating...
      qiime2.jupyter  OK


# 데이터 다운로드

In [2]:
!wget https://data.qiime2.org/2018.8/tutorials/fmt/sample_metadata.tsv
!wget https://data.qiime2.org/2018.8/tutorials/fmt/fmt-tutorial-demux-1-10p.qza
!wget https://data.qiime2.org/2018.8/tutorials/fmt/fmt-tutorial-demux-2-10p.qza

--2019-10-29 16:32:34--  https://data.qiime2.org/2018.8/tutorials/fmt/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/1SEtU6raRGD4GDV74x04zhEHKGA1NcdX1Iv5btIVKQiA/export?gid=0&format=tsv [following]
--2019-10-29 16:32:35--  https://docs.google.com/spreadsheets/d/1SEtU6raRGD4GDV74x04zhEHKGA1NcdX1Iv5btIVKQiA/export?gid=0&format=tsv
Resolving docs.google.com (docs.google.com)... 172.217.25.14, 2404:6800:4005:809::200e
Connecting to docs.google.com (docs.google.com)|172.217.25.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: 'sample_metadata.tsv'

    [ <=>                                   ] 16,026      --.-K/s   in 0.01s   

2019-10-29 16:32:37 (1.07 MB/s) - 'sample_metadata.tsv' saved [16026]

--2019-1

In [3]:
import pandas as pd
from qiime2.metadata.metadata import Metadata
from qiime2.metadata.metadata import CategoricalMetadataColumn
from qiime2.sdk import Artifact
from qiime2.sdk import PluginManager
from qiime2.sdk import Result

pm = PluginManager()
demux_plugin = pm.plugins['demux']
demux_summarize = demux_plugin.actions['summarize']

print(pm.plugins)

{'vsearch': <qiime2.plugin.plugin.Plugin object at 0x7f113c437710>, 'types': <qiime2.plugin.plugin.Plugin object at 0x7f113c4d9898>, 'taxa': <qiime2.plugin.plugin.Plugin object at 0x7f113c45b898>, 'sample-classifier': <qiime2.plugin.plugin.Plugin object at 0x7f11183130b8>, 'quality-filter': <qiime2.plugin.plugin.Plugin object at 0x7f11182a8ef0>, 'quality-control': <qiime2.plugin.plugin.Plugin object at 0x7f11182e34e0>, 'phylogeny': <qiime2.plugin.plugin.Plugin object at 0x7f11181d5438>, 'metadata': <qiime2.plugin.plugin.Plugin object at 0x7f111826efd0>, 'longitudinal': <qiime2.plugin.plugin.Plugin object at 0x7f1118324b00>, 'gneiss': <qiime2.plugin.plugin.Plugin object at 0x7f1116fa16a0>, 'fragment-insertion': <qiime2.plugin.plugin.Plugin object at 0x7f1116975978>, 'feature-table': <qiime2.plugin.plugin.Plugin object at 0x7f11168d16a0>, 'feature-classifier': <qiime2.plugin.plugin.Plugin object at 0x7f11168adc88>, 'emperor': <qiime2.plugin.plugin.Plugin object at 0x7f1116882f28>, 'diver

# Demultiplexing

In [4]:
print(demux_summarize.description)
demux_summarize_signature = demux_summarize.signature
print(demux_summarize_signature.inputs)
print(demux_summarize_signature.parameters)
print(demux_summarize_signature.outputs)

Summarize counts per sample for all samples, and generate interactive positional quality plots based on `n` randomly selected sequences.
OrderedDict([('data', ParameterSpec(qiime_type=SampleData[SequencesWithQuality | PairedEndSequencesWithQuality | JoinedSequencesWithQuality], view_type=<class 'q2_demux._summarize._visualizer._PlotQualView'>, default=NOVALUE, description='The demultiplexed sequences to be summarized.'))])
OrderedDict([('n', ParameterSpec(qiime_type=Int, view_type=<class 'int'>, default=10000, description='The number of sequences that should be selected at random for quality score plots. The quality plots will present the average positional qualities across all of the sequences selected. If input sequences are paired end, plots will be generated for both forward and reverse reads for the same `n` sequences.'))])
OrderedDict([('visualization', ParameterSpec(qiime_type=Visualization, view_type=None, default=NOVALUE, description=NOVALUE))])


In [5]:
seqs1 = Result.load('fmt-tutorial-demux-1-10p.qza')
sum_data1 = demux_summarize(seqs1)
sum_data1.visualization

<visualization: Visualization uuid: 7b20ce6c-2466-46d2-91da-fad2e6d91e0a>

In [6]:
seqs2 = Result.load('fmt-tutorial-demux-2-10p.qza')
sum_data2 = demux_summarize(seqs2)
sum_data2.visualization

<visualization: Visualization uuid: 74cd6426-4c09-4291-9007-62e6917ae17f>

In [7]:
dada2_plugin = pm.plugins['dada2']
dada2_denoise_single = dada2_plugin.actions[
    'denoise_single']
qual_control1 = dada2_denoise_single(
    demultiplexed_seqs=seqs1,
    trunc_len=150,
    trim_left=13)
qual_control2 = dada2_denoise_single(
    demultiplexed_seqs=seqs2,
    trunc_len=150,
    trim_left=13)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-vokc5qk2/5db90b62-c6e4-4c09-8c79-c0cdfbe2cea0/data /tmp/tmpc9cr54av/output.tsv.biom /tmp/tmpc9cr54av/track.tsv /tmp/tmpc9cr54av 150 13 2.0 2 Inf consensus 1.0 1 1000000 NULL 16

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/qiime2-archive-nnx_4koe/5f459ea8-c6a8-438e-b42f-35e4f86166d7/data /tmp/tmpi5k5k27x/output.tsv.biom /tmp/tmpi5k5k27x/track.tsv /tmp/tmpi5k5k27x 150 13 2.0 2 Inf consensus 1.0 1 1000000 NULL 16



In [8]:
metadata_plugin = pm.plugins['metadata']
metadata_tabulate = metadata_plugin.actions[
    'tabulate']
stats_meta1 = metadata_tabulate(
    input=qual_control1.denoising_stats.view(Metadata))
stats_meta1.visualization

<visualization: Visualization uuid: 4b5e54b9-cb0a-448b-8219-190b2e8ca4ff>

In [9]:
stats_meta2 = metadata_tabulate(
    input=qual_control2.denoising_stats.view(Metadata))
stats_meta2.visualization

<visualization: Visualization uuid: b13017d5-e29b-4139-b624-4af90e600795>

In [10]:
ft_plugin = pm.plugins['feature-table']
ft_merge = ft_plugin.actions['merge']
ft_merge_seqs = ft_plugin.actions['merge_seqs']
ft_summarize = ft_plugin.actions['summarize']
ft_tab_seqs = ft_plugin.actions['tabulate_seqs']

table_merge = ft_merge(
    tables=[qual_control1.table,qual_control2.table])
seqs_merge = ft_merge_seqs(
    data=[qual_control1.representative_sequences,
    qual_control2.representative_sequences])

In [11]:
ft_sum = ft_summarize(table=table_merge.merged_table)
ft_sum.visualization

/home/partrita/miniconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:122: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'sample-frequency-detail.csv'))
/home/partrita/miniconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/q2_feature_table/_summarize/_visualizer.py:124: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  os.path.join(output_dir, 'feature-frequency-detail.csv'))


<visualization: Visualization uuid: 0d63af01-c36e-4390-b235-415606bdc5a0>

In [12]:
tab_seqs = ft_tab_seqs(data=seqs_merge.merged_data)
tab_seqs.visualization

<visualization: Visualization uuid: fdd84b53-f0b3-4a67-ae9e-0ef2f1c11587>